In [4]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = 'cpu'
print("Device:", device)

Device: cpu


In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe_translate = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", device=device)

test_text = "Bonjour, je suis Quentin, comment allez-vous aujourd'hui ? J'ai un problème médical dont j'aimerai discuter."
translation = pipe_translate(test_text)
print("Translation:", translation)

Translation: [{'translation_text': "Hello, I'm Quentin, how are you today? I have a medical problem I'd like to discuss."}]


In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# pipe = pipeline("text-to-speech", model="suno/bark-small", device=device)
# pipe = pipeline("text-to-speech", model="suno/bark", device=device)
pipe_tts = pipeline("text-to-speech", model="facebook/mms-tts-eng", device=device) # Better quality, way faster than bark

speech = pipe_tts(translation[0]["translation_text"])

import IPython
IPython.display.Audio(speech["audio"], rate=speech["sampling_rate"])

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [7]:
import gradio as gr
from transformers import pipeline
import torch
import soundfile as sf
import numpy as np
import scipy.io.wavfile as wavfile

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = 'cpu'
print("Device:", device)

pipe_translate = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", device=device)
pipe_tts = pipeline("text-to-speech", model="facebook/mms-tts-eng", device=device) # Better quality, way faster than bark

def get_translation(text):
    return pipe_translate(text)[0]["translation_text"]

def get_audio(text):
    return speech["sampling_rate"], (speech["audio"]* 32767).astype(np.int16).T

with gr.Blocks() as demo:    
    input_text = gr.Textbox(
                label="Input text",
                info="Your text",
                lines=3,
                placeholder="Écrire le texte à traduire",
            )

    
    
    translation_button = gr.Button("Traduire...")
    output_text = gr.Textbox(
                label="Output text",
                info="Your text",
                lines=3,
                placeholder="Votre traduction",
            )
    speech_button = gr.Button("Générer audio...")
    translation_button.click(
            get_translation,
            inputs=[
                input_text
            ],
            outputs=[
                output_text
            ],
    )
    speech_button.click(
            get_audio,
            inputs=[
                output_text
            ],
            outputs=[
                gr.Audio(label="Output")
            ],
    )

demo.launch()

Device: cpu


/home/qbateux/pytorch/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wav

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
